In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
import sys
import seaborn as sns
import swifter
sys.path.append('../')
pd.options.mode.chained_assignment = None 



In [2]:
#combine all CSV's of the same data
files = open("bees/files-short.txt")
filenames = []
while True:
    line = files.readline().strip()
    if not line:
        break
    filenames.append("bees/" + line)

vdf = pd.concat(map(pd.read_csv, filenames), ignore_index=True)

In [3]:
vdf['track_tagid'] = vdf['track_tagid'].swifter.apply(lambda x: str(x))
vdf = vdf.sort_values(by=['track_tagid','track_starttime']).reset_index()




Pandas Apply:   0%|          | 0/71422 [00:00<?, ?it/s]

In [5]:
vdf['diff'] = vdf['track_id'].diff().shift(periods=-1)
vdf['breakpoints'] = vdf['diff'] != 0

In [15]:
breakpoints = vdf[vdf['breakpoints'] == True].index.tolist()
vdf['event_id'] = np.nan
first_detection = 0
for i in range(len(breakpoints)):
        
        vdf['event_id'].iloc[first_detection:breakpoints[i]+1]
        first_detection = breakpoints[i]+1

In [17]:
vdf['event_id'] = vdf['event_id'].swifter.apply(lambda x: int(x))

Pandas Apply:   0%|          | 0/71422 [00:00<?, ?it/s]

### GRAPH GENERATION

In [19]:
for i in vdf['event_id'].unique():
    detections = vdf[vdf['event_id'] == i]

    positions = detections['cy']
    x = detections['frame']
    plot = sns.lineplot(x=x,y=positions)
    plot.set_ylim(-100, 1500)
    plt = plot.get_figure()
    plt.savefig(f"graphs/{i}.png")
    plt.clf()


<Figure size 640x480 with 0 Axes>

In [20]:
vdf.to_csv("beeActivity.csv")